In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [2]:
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [3]:
question = """
    Classes A and B start from the school at the same time to go to a park that is 51 km away. \
    Class A walks at a speed of 4 km/h, and Class B walks at a speed of 5 km/h. \
    The school has a car that can travel at 60 km/h and can carry one class of students at a time. \
    To ensure that both classes arrive at the park in the shortest time possible, how many kilometres do Classes A and B need to walk?
"""

In [4]:
prompt_one = ChatPromptTemplate.from_template(
    "What is the mathmatical equation of the {question}?"
)

chain_one = LLMChain(llm=llm, prompt=prompt_one)

In [5]:
prompt_two = ChatPromptTemplate.from_template(
    "what is the solution of the :{equation}"
)

chain_two = LLMChain(llm=llm, prompt=prompt_two)

In [6]:
overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [9]:
overall_chain.run(question)



> Entering new SimpleSequentialChain chain...
Let's assume that both classes start walking at the same time and meet at some point during their journey. Let's say they meet after x kilometers.

The time taken by Class A to walk x kilometers is given by:
Time taken by Class A = Distance / Speed = x / 4

The time taken by Class B to walk (51 - x) kilometers is given by:
Time taken by Class B = Distance / Speed = (51 - x) / 5

Since the school has a car that can carry one class at a time, it will have to make two trips. The time taken by the car to travel x kilometers is given by:
Time taken by the car = Distance / Speed = x / 60

To ensure that both classes arrive at the park in the shortest time possible, the total time taken by both classes and the car should be minimized. Therefore, we need to minimize the sum of the times taken by Class A, Class B, and the car.

Total time = Time taken by Class A + Time taken by Class B + Time taken by the car
Total time = x / 4 + (51 - x) / 5 + x 

'The solution is that both classes will have to walk the entire distance of 51 km to ensure that they arrive at the park in the shortest time possible.'

In [7]:
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

In [8]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    }
]

In [10]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [11]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [12]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [13]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [14]:
chain = MultiPromptChain(router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain, verbose=True
)

In [15]:
chain.run(question)



> Entering new MultiPromptChain chain...


/home/boringtao/Conda/envs/gpt/lib/python3.9/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'Classes A and B start from the school at the same time to go to a park that is 51 km away. Class A walks at a speed of 4 km/h, and Class B walks at a speed of 5 km/h. The school has a car that can travel at 60 km/h and can carry one class of students at a time. To ensure that both classes arrive at the park in the shortest time possible, how many kilometres do Classes A and B need to walk?'}
> Finished chain.


"To solve this problem, we need to consider the time it takes for each class to walk and the time it takes for the car to travel.\n\nLet's start by calculating the time it takes for Class A to walk to the park. We can use the formula: time = distance / speed.\n\nFor Class A:\nTime taken = 51 km / 4 km/h = 12.75 hours\n\nNow, let's calculate the time it takes for Class B to walk to the park:\nTime taken = 51 km / 5 km/h = 10.2 hours\n\nSince the car can only carry one class at a time, it will have to make two trips to transport both classes. The total time taken by the car will be the sum of the time taken for each trip.\n\nFor the first trip:\nTime taken = distance / speed = 51 km / 60 km/h = 0.85 hours\n\nFor the second trip:\nTime taken = distance / speed = 51 km / 60 km/h = 0.85 hours\n\nTherefore, the total time taken by the car is 0.85 + 0.85 = 1.7 hours.\n\nTo ensure that both classes arrive at the park in the shortest time possible, we need to minimize the total time. Since the 